<a href="https://colab.research.google.com/github/immortalRegis/Data-Analysis/blob/main/Database_with_SQL_and_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

In [2]:
dbconn = sqlite3.connect('newdb')
cur = dbconn.cursor()

We create a connection to the database and a cursor object.

In [3]:
cur.execute("""CREATE TABLE IF NOT EXISTS booktable(id INTEGER PRIMARY KEY, title TEXT, author TEXT, price TEXT, year TEXT)""")
dbconn.commit()

In [4]:
cur.execute("""INSERT INTO booktable VALUES (1, "The Cockroach", "Franz Kafka", 3.50, 1880),(2, "The Brothers Karamazov", "Fyodor Dostoyevsky", 4.00, 1900)""")
dbconn.commit()

We run some SQL commands using the cursor.execute() method and commit changes to the database. We then load data from the database into dataframes in order to perform analysis easily.

In [5]:
bookdf = pd.read_sql_query("SELECT * FROM booktable",dbconn, index_col = 'id')
bookdf

,title,author,price,year
id,,,,
1,The Cockroach,Franz Kafka,3.5,1880
2,The Brothers Karamazov,Fyodor Dostoyevsky,4.0,1900


In [6]:
cur.execute("""INSERT INTO booktable VALUES(3, "The Passport of Mallam Ilia", "Cyprian Ekwensi", 5.00 ,1960)""")
cur.execute("""UPDATE booktable SET year = 1880 WHERE title LIKE 'The brother'""")
cur.execute("""UPDATE booktable SET title = 'The Metamorphosis', year = 1915 WHERE author = 'Franz Kafka'""")
dbconn.commit()

In [7]:
bookdf

,title,author,price,year
id,,,,
1,The Cockroach,Franz Kafka,3.5,1880
2,The Brothers Karamazov,Fyodor Dostoyevsky,4.0,1900


In [8]:
bookdf = pd.read_sql_query("SELECT * FROM booktable",dbconn, index_col = 'id')
bookdf

,title,author,price,year
id,,,,
1,The Metamorphosis,Franz Kafka,3.5,1915
2,The Brothers Karamazov,Fyodor Dostoyevsky,4.0,1900
3,The Passport of Mallam Ilia,Cyprian Ekwensi,5.0,1960


In [21]:
dbconn = sqlite3.connect('chinook.db')
cur = dbconn.cursor()

We try some more advanced queries using tables from the chinook database.

First we perform some EDA on the database to investigate the nature of the tables present.

In [15]:
tablelist = cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table'""").fetchall()
#get list of tables in the database
tablelist

[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',)]

In [16]:
albumdf = pd.read_sql_query("""SELECT * FROM albums""", dbconn)
albumdf.head()


,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [17]:
artistdf = pd.read_sql_query("""SELECT * FROM artists""", dbconn)
artistdf.head()


,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [18]:
query = """
    SELECT al.artistid as id, ar.name as artistname, al.title as title
    FROM albums al JOIN artists ar ON al.artistid = ar.artistid
    WHERE al.artistid BETWEEN 1 AND 10 
    ORDER by id
    LIMIT 15 
"""
albartdf = pd.read_sql_query(query, dbconn, index_col = 'id')
albartdf

,artistname,title
id,,
1,AC/DC,For Those About To Rock We Salute You
1,AC/DC,Let There Be Rock
2,Accept,Balls to the Wall
2,Accept,Restless and Wild
3,Aerosmith,Big Ones
4,Alanis Morissette,Jagged Little Pill
5,Alice In Chains,Facelift
6,Antônio Carlos Jobim,Warner 25 Anos
6,Antônio Carlos Jobim,Chill: Brazil (Disc 2)


In [22]:
cur.close()
dbconn.close()
